In [ ]:
!apt update && apt-get install -y libnetcdf-dev libgd-dev
!mkdir -p glmpy
!git clone --depth 1 --filter=blob:none --sparse https://github.com/AquaticEcoDynamics/efi-workshop
!cd efi-workshop && git sparse-checkout set glmpy case_studies && cp -r glmpy ../ && cp -r case_studies ../ && cd .. && rm -rf efi-workshop
!curl https://raw.githubusercontent.com/AquaticEcoDynamics/efi-workshop/refs/heads/main/requirements.txt --output requirements.txt
!pip install -r requirements.txt
!curl https://raw.githubusercontent.com/AquaticEcoDynamics/efi-workshop/refs/heads/main/bin/glm_3.3.3_linux_x86_64 --output glm
!chmod +x glm
!pip install -e glmpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from glmpy import plots
from glmpy.nml import nml, glm_nml, aed_nml
from glmpy.example_sims import SparklingSim

### Running `SparklingSim`

glm-py's `GLMSim` objects represent a GLM simulation. They are responsible for storing all the model data (parameters and boundary condition data) and providing methods needed to run and modify the simulation. The `example_sims` module provides prebuilt `GLMSim` objects that can be run immediately. Initialise an instance of the `SparklingSim` class to get started:  

In [ ]:
sparkling = SparklingSim(sim_name="sparkling")

Running the simulation is as simple as calling the `run()` method:

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

### Plotting outputs

glm-py's `plots` module provides a collection of classes for plotting GLM outputs with matplotlib. The `LakePlotter` class provides methods for plotting GLM's `lake.csv` output. Initialise an instance of `LakePlotter` by providing the path to the `lake.csv` output. By defualt glmpy will create a output directory that matches the `sim_name`:

In [ ]:
lake = plots.LakePlotter("sparkling/output/lake.csv")

The `lake_volume()` method is used for creating a line plot of the lake volume (in m^3). Call the method by providing a matplotlib `Axes` object to plot on:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
lake.lake_volume(ax=ax)

`water_balance_components()` is used to create a daily line plot of the water balance components (m^3/day):

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
out = lake.water_balance_components(
    ax=ax, 
    rain_params={"linestyle": "--"},
    local_runoff_params=None, 
    overflow_vol_params=None,
    snowfall_params=None
)
ax.legend(handles=out)

More advanced plots can be created using `LakePlotter`'s various mathod and the full matplotlib API:

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 15))
date_formatter = mdates.DateFormatter("%m/%y")

out = lake.water_balance(ax=ax[0, 0])
ax[0, 0].xaxis.set_major_formatter(date_formatter)
out = lake.water_balance_components(
    ax=ax[0,1], 
    local_runoff_params=None, 
    overflow_vol_params=None,
    snowfall_params=None
)
ax[0, 1].legend(handles=out, ncols=2, loc=0)
ax[0, 1].xaxis.set_major_formatter(date_formatter)
out = lake.lake_temp(ax[1, 0])
ax[1, 0].legend(handles=out, ncols=2, loc=0)
ax[1, 0].xaxis.set_major_formatter(date_formatter)
out = lake.heat_balance_components(ax[1, 1])
ax[1, 1].legend(handles=out, ncols=2, loc=0)
ax[1, 1].xaxis.set_major_formatter(date_formatter)

The `NCProfile` class of the `plots` module can be used plot an output variable for all depths and timesteps of the simulation. This class is initialised by providing a path to the `output.nc` NetCDF file:

In [ ]:
nc = plots.NCProfile("sparkling/output/output.nc")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
out = nc.plot_var(ax=ax, var="temp")
col_bar = fig.colorbar(out)
col_bar.set_label("Temperature (°C)")

`NCProfile` also provides methods to assist with automating the plotting of variables:

- `get_vars` returns a list of variables that can be plotted with plot_var
- `get_long_name` returns the unabbreviated name of a variable
- `get_units` returns the units of a variable

In [ ]:
vars = nc.get_vars()
vars

In [ ]:
plot_vars = vars[3:5] # returns ['salt', 'temp']
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
for idx, var, in enumerate(plot_vars):
    out = nc.plot_var(axs[idx], var)
    long_name = nc.get_long_name(var)
    units = nc.get_units(var)
    col_bar = fig.colorbar(out)
    col_bar.set_label(f"{long_name} ({units})")

### Inspecting the NML

The `nml` attribute of a `GLMSim` object is resposible for storing all the model parameters found in `.nml` files. These parameters are represented as `NMLParam` objects that are stored in a hierarchical structure of `NMLBlock` objects and `NML` objects. Individual parameters can be accessed directly on the `GLMSim` object using the following syntax:

In [ ]:
sparkling.nml["glm"].blocks["glm_setup"].params["min_layer_vol"].value

In [ ]:
sparkling.nml["glm"].blocks["glm_setup"].params["min_layer_vol"].value = 1.0

For a less verbose syntax, parameter values can also be accessed using the `get_param_value()` and `set_param_value()` methods:

In [ ]:
sparkling.set_param_value("glm", "glm_setup", "min_layer_vol", 0.5)

In [ ]:
sparkling.get_param_value("glm", "glm_setup", "min_layer_vol")

Blocks of parameters (`NMLBlock` objects) can also be accessed directly via attributes or using the `get_block()` and `set_block()` methods. The `to_dict()` method returns a dictionary representation of the block's parameters:

In [ ]:
sparkling.nml["glm"].blocks["glm_setup"].to_dict()

In [ ]:
sparkling.get_block("glm", "glm_setup").to_dict()

`NML` objects, that store a collection of `NMLBlock` objects, can be accessed in a similar way:

In [ ]:
sparkling.nml["glm"].to_dict()

In [ ]:
sparkling.get_nml("glm").to_dict()

### Inspecting the boundary conditions

All boundary conditions files needed to run a GLM simulation are stored in the `bcs` attribute. `bcs` is a dictionary where the key is the output file name (without the file extension), and the value is a Pandas `DataFrame` object:

In [ ]:
sparkling.bcs['nldas_driver']

Boundary condition dataframes can be modified using the Pandas API:

In [ ]:
original_bcs = sparkling.bcs['nldas_driver']
modified_bcs = sparkling.bcs['nldas_driver']
modified_bcs["ShortWave"] = modified_bcs["ShortWave"] * 2
sparkling.bcs['nldas_driver'] = modified_bcs
sparkling.bcs['nldas_driver']

In [ ]:
sparkling.bcs['nldas_driver'] = original_bcs

### Adding AED

`SparklingSim` currently lacks the `wq_setup` block:

In [ ]:
print(sparkling.get_block(nml_name="glm", block_name="wq_setup"))

We can create a `NMLBlock` object for `wq_setup` using the `WQSetupBlock` class in the `glm_nml` module:

In [ ]:
wq_setup = glm_nml.WQSetupBlock(
    wq_lib="aed",
    wq_nml_file='aed/aed.nml',
    ode_method=1,
    split_factor=1,
    bioshade_feedback=True,
    repair_state=True,
)

Next, assign the block to the `GLMNML` object:

In [ ]:
sparkling.set_block(nml_name="glm", block=wq_setup)

### Case 1

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case1.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.set_nml(aed_nml_obj)
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")

### Case 2

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case2.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.set_nml(aed_nml_obj)
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")

### Case 3

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case3.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.set_nml(aed_nml_obj)
sparkling.aed_dbase = ["case_studies/aed_phyto_pars.csv"]
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")

### Case 4

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case4.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.nml["aed"] = aed_nml_obj
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")

### Case 5

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case5.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.nml["aed"] = aed_nml_obj
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")

### Case 6

In [ ]:
nml_file = nml.NMLReader("case_studies/aed_case6.json")
aed_nml_obj = nml_file.to_nml_obj(aed_nml.AEDNML)
sparkling.set_nml(aed_nml_obj)
sparkling.aed_dbase = [
    "case_studies/aed_phyto_pars.csv", "case_studies/aed_zoop_pars.csv"
]
sparkling.get_param_value("aed", "aed_models", "models")

In [ ]:
sparkling.run(time_sim=True, write_log=True, glm_path="bin/glm_3.3.3_linux_arm64")

In [ ]:
nc.get_vars()

In [ ]:
var = "OXY_oxy"
fig, ax = plt.subplots(figsize=(10, 5))
plot_params = {"vmin": 0, "vmax": 550}
out = nc.plot_var(ax=ax, var=var, param_dict=plot_params)
col_bar = fig.colorbar(out)

var_long_name = nc.get_long_name(var)
var_units = nc.get_units(var)
col_bar.set_label(f"{var_long_name} ({var_units})")